In [8]:
import argparse
import chromadb
import pysbd
import uuid
from embedding_functions import get_embedding_fn
from utils import load_dataset

In [3]:
model = "BAAI/bge-small-en"
device = "mps"
embedding_fn = get_embedding_fn(model, device, normalize=False)
print(embedding_fn)

In [7]:
data = load_dataset('data/processed_for_chroma/reviews/Astro_Reviews.json')
print(type(data))
print(len(data))

<class 'list'>
996


In [ ]:
# record -> chunking -> use embdding_fn to create insertion records -> insert into db

In [9]:
# Create a sentence chunker
SEG = pysbd.Segmenter(language="en", clean=False)
def abstract_sentence_chunker(record, embedding_fn):
    # Data preparation
    abstract = record['abstract']
    sentences = SEG.segment(record['body'])

    # Construct the texts to be embedded
    texts = [abstract + '\n' + sentence for sentence in sentences]
    vectors = embedding_fn(texts)

    return zip(sentences, vectors)

# Create an (abstract, sentence) chunker

In [22]:
def augmented_sentence_embeddings(record, embedding_fn, augmenting_fn):
    sentences = SEG.segment(record['body'])
    texts = augmenting_fn(record, sentences)
    vectors = embedding_fn(texts)
    return list(zip(sentences, vectors))

def augment_with_abstract(record, sentences):
    abstract = record['abstract']
    return [abstract + '\n' + sentence for sentence in sentences]

def no_augmentation(record, sentences):
    return sentences

In [20]:
abstract_and_sentence_embeddings = augmented_sentence_embeddings(data[0], embedding_fn, augment_with_abstract)

In [23]:
sentence_only_embeddings = augmented_sentence_embeddings(data[0], embedding_fn, no_augmentation)

In [24]:
print(f"Lenght of abstract_and_sentence_embeddings: {len(abstract_and_sentence_embeddings)}")
print(f"Lenght of sentence_only_embeddings: {len(sentence_only_embeddings)}")

Lenght of abstract_and_sentence_embeddings: 699
Lenght of sentence_only_embeddings: 699
